In [13]:
import csv
from bs4 import BeautifulSoup

In [16]:
# for chrome
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [17]:
#Startup the driver
driver = webdriver.Chrome()

In [18]:
url = 'https://www.amazon.in'
driver.get(url)

In [19]:
def get_url(search):
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_ss_ts-doa-p_1_7'
    search_term = search.replace(' ', '+')
    return template.format(search_term)

In [20]:
url = get_url('self help book')
print(url)

https://www.amazon.in/s?k=self+help+book&ref=nb_sb_ss_ts-doa-p_1_7


In [21]:
driver.get(url)

In [22]:
#extracting collection
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [23]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [24]:
len(results)

22

In [25]:
# PROTOTYPE THE RECORD
item = results[0]

In [26]:
atag = item.h2.a

In [28]:
description = atag.text.strip()

'World’s Greatest Books For Personal Growth & Wealth (Set of 4 Books): Perfect Motivational Gift Set'

In [30]:
url = 'https://www.amazon.in' + atag.get('href')

In [31]:
price_parent = item.find('span', 'a-price')

In [34]:
price = price_parent.find('span', 'a-offscreen').text

In [36]:
rating = item.i.text

'4.5 out of 5 stars'

In [43]:
#Generalizing pattern
def extract_record(item):
    #Extract and return data from a single record
    
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    
    #price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
    
    #ratings
    rating = item.i.text
    
    result = (description, price, rating, url)
    
    return result

In [45]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [50]:
for row in records:
    print(row[1])  ##TESTING

₹299
₹399
₹313
₹99
₹157
₹297
₹445
₹59
₹193
₹299
₹212
₹99
₹259
₹96.25
₹99
₹287
₹148
₹171
₹299
₹299
₹139
₹287


In [51]:
#Navigating to next page
def get_url(search):
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_ss_ts-doa-p_1_7'
    search_term = search.replace(' ', '+')
    
    #adding term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url

In [56]:
#Putting it all together
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def get_url(search):
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_ss_ts-doa-p_1_7'
    search_term = search.replace(' ', '+')
    
    #adding term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url

def extract_record(item):
    #Extract and return data from a single record
    
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')
    
    #price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
    
    #ratings
    rating = item.i.text
    
    result = (description, price, rating, url)
    
    return result

def main(search_term):
    
    #starting the webdriver
    driver = webdriver.Chrome()
    
    record = []
    url = get_url(search_term)
    
    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
            
    driver.close() #closing of driver
    
    #saving the data to csv file
    with open('results.csv', 'w', newline = '', encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description','Price','Rating','Url'])
        writer.writerows(records)

In [57]:
main('self help book')